In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame as df
import re
import matplotlib.pyplot as plt
import mapclassify
import folium
import geopandas

plt.style.use(style="seaborn")
%matplotlib inline

In [2]:
Data = pd.read_csv('Data.csv', index_col=0)
reviews = pd.read_csv('ReviewsRaw.csv')

In [3]:
Data

,name,vicinity,type/0,type/1,type/2,type/3,type/4,geometry/location/lat,geometry/location/lng,geometry/viewport/southwest/lat,...,bar,bowling_alley,movie_theater,type/10,type/11,type/12,museum,casino,PlaceSearchResponse/status,Unnamed: 0.1
0,Tate Britain,"Millbank, London",art_gallery,tourist_attraction,museum,point_of_interest,establishment,51.491062,-0.127789,51.489506,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Institute of Contemporary Arts,"The Mall, London",art_gallery,tourist_attraction,movie_theater,point_of_interest,establishment,51.506333,-0.130962,51.504714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Institute of International Visual Arts,"16 John Islip Street, London",art_gallery,library,point_of_interest,establishment,NaN,51.490483,-0.129298,51.489172,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Beaconsfield Contemporary Art,"Beaconsfield Ltd, 22 Newport Street, London",art_gallery,point_of_interest,establishment,NaN,NaN,51.492096,-0.118686,51.490715,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Enchanted Group,"Orange Studio, Old Paradise, 20 Carlisle Lane,...",art_gallery,cafe,electronics_store,home_goods_store,point_of_interest,51.498503,-0.116619,51.497200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3635,Revolution Beaconsfield,"Maxwell Rd, Beaconsfield",bar,restaurant,food,point_of_interest,establishment,51.610356,-0.643456,51.608954,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,860.0
3636,Lizard Lounge,"186-188 George Ln, London",bar,restaurant,food,point_of_interest,establishment,51.594563,0.023954,51.593240,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,861.0
3637,Popworld Guildford,"2-3 Armour Buildings, Guildford",night_club,bar,point_of_interest,establishment,NaN,51.236479,-0.577977,51.235205,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,862.0
3638,64th & Social,"64 Clapham High St, London",bar,point_of_interest,establishment,NaN,NaN,51.463736,-0.133134,51.462325,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,881.0


In [4]:
Data.columns

Index(['name', 'vicinity', 'type/0', 'type/1', 'type/2', 'type/3', 'type/4',
       'geometry/location/lat', 'geometry/location/lng',
       'geometry/viewport/southwest/lat', 'geometry/viewport/southwest/lng',
       'geometry/viewport/northeast/lat', 'geometry/viewport/northeast/lng',
       'rating', 'icon', 'reference', 'opening_hours/open_now',
       'photo/photo_reference', 'photo/width', 'photo/height',
       'photo/html_attribution', 'user_ratings_total', 'place_id', 'scope',
       'plus_code/global_code', 'plus_code/compound_code',
       'icon_background_color', 'business_status', 'icon_mask_base_uri',
       'type/5', 'type/6', 'type/7', 'permanently_closed', 'price_level',
       'type/8', 'type/9', 'bar', 'bowling_alley', 'movie_theater', 'type/10',
       'type/11', 'type/12', 'museum', 'casino', 'PlaceSearchResponse/status',
       'Unnamed: 0.1'],
      dtype='object')

In [5]:
reviews

,result/address_components/0/long_name,result/address_components/0/short_name,result/address_components/0/types/0,result/address_components/1/long_name,result/address_components/1/short_name,result/address_components/1/types/0,result/address_components/2/long_name,result/address_components/2/short_name,result/address_components/2/types/0,result/address_components/2/types/1,...,result/address_components/1/types/2,result/address_components/8/types/1,result/address_components/9/long_name,result/address_components/9/short_name,result/address_components/9/types/0,result/address_components/4/types/2,result/address_components/9/types/1,result/address_components/10/long_name,result/address_components/10/short_name,result/address_components/10/types/0
0,Millbank,Millbank,route,London,London,postal_town,Greater London,Greater London,administrative_area_level_2,political,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,The Mall,The Mall,route,St. James's,St. James's,neighborhood,London,London,postal_town,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,16,street_number,John Islip Street,John Islip St,route,London,London,postal_town,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Beaconsfield Ltd,Beaconsfield Ltd,premise,22,22,street_number,Newport Street,Newport St,route,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,London,London,postal_town,Greater London,Greater London,administrative_area_level_2,England,England,administrative_area_level_1,political,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3634,Maxwell Road,Maxwell Rd,route,Beaconsfield,Beaconsfield,postal_town,Buckinghamshire,Buckinghamshire,administrative_area_level_2,political,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3635,186-188,186-188,street_number,George Lane,George Ln,route,London,London,postal_town,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3636,2-3,2-3,street_number,Armour Buildings,Armour Buildings,route,Guildford,Guildford,postal_town,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3637,64,64,street_number,Clapham High Street,Clapham High St,route,London,London,postal_town,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
reviews.columns

Index(['result/address_components/0/long_name',
       'result/address_components/0/short_name',
       'result/address_components/0/types/0',
       'result/address_components/1/long_name',
       'result/address_components/1/short_name',
       'result/address_components/1/types/0',
       'result/address_components/2/long_name',
       'result/address_components/2/short_name',
       'result/address_components/2/types/0',
       'result/address_components/2/types/1',
       ...
       'result/address_components/1/types/2',
       'result/address_components/8/types/1',
       'result/address_components/9/long_name',
       'result/address_components/9/short_name',
       'result/address_components/9/types/0',
       'result/address_components/4/types/2',
       'result/address_components/9/types/1',
       'result/address_components/10/long_name',
       'result/address_components/10/short_name',
       'result/address_components/10/types/0'],
      dtype='object', length=242)

Initial work on the main data

In [7]:
# Dropping revised unnecessary columns
DataV2 = Data.drop(['icon','reference','opening_hours/open_now','photo/photo_reference',
                    'photo/width','photo/height','photo/html_attribution',
                    'scope','plus_code/global_code','plus_code/compound_code',
                    'icon_background_color','business_status','icon_mask_base_uri','permanently_closed',
                    'bar','bowling_alley','movie_theater','museum','casino','PlaceSearchResponse/status','Unnamed: 0.1']
                   , axis=1)
# Checking the nulls in types 0 to 12
print('type 0: ',DataV2['type/0'].isna().sum())
print('type 1: ',DataV2['type/1'].isna().sum())
print('type 2: ',DataV2['type/2'].isna().sum())
print('type 3: ',DataV2['type/3'].isna().sum())
print('type 4: ',DataV2['type/4'].isna().sum())
print('type 5: ',DataV2['type/5'].isna().sum())
print('type 6: ',DataV2['type/6'].isna().sum())
print('type 7: ',DataV2['type/7'].isna().sum())
print('type 8: ',DataV2['type/8'].isna().sum())
print('type 9: ',DataV2['type/9'].isna().sum())
print('type 10: ',DataV2['type/10'].isna().sum())
print('type 11: ',DataV2['type/11'].isna().sum())
print('type 12: ',DataV2['type/12'].isna().sum())

#No types will be removed

#Removing Viewport geo tags
DataV2 = DataV2.drop(['geometry/viewport/southwest/lat','geometry/viewport/southwest/lng',
                      'geometry/viewport/northeast/lat','geometry/viewport/northeast/lng'], axis=1)
DataV2

type 0:  0
type 1:  0
type 2:  149
type 3:  1586
type 4:  2591
type 5:  3218
type 6:  3492
type 7:  3581
type 8:  3616
type 9:  3626
type 10:  3632
type 11:  3633
type 12:  3636


,name,vicinity,type/0,type/1,type/2,type/3,type/4,geometry/location/lat,geometry/location/lng,rating,...,place_id,type/5,type/6,type/7,price_level,type/8,type/9,type/10,type/11,type/12
0,Tate Britain,"Millbank, London",art_gallery,tourist_attraction,museum,point_of_interest,establishment,51.491062,-0.127789,4.7,...,ChIJvWjCxekEdkgRoCgQVJHZH_U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Institute of Contemporary Arts,"The Mall, London",art_gallery,tourist_attraction,movie_theater,point_of_interest,establishment,51.506333,-0.130962,4.4,...,ChIJpwrHSNAEdkgR1YO1RjflYXw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Institute of International Visual Arts,"16 John Islip Street, London",art_gallery,library,point_of_interest,establishment,NaN,51.490483,-0.129298,5.0,...,ChIJrxGsi7ocdkgRfYUYz7mhuMs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Beaconsfield Contemporary Art,"Beaconsfield Ltd, 22 Newport Street, London",art_gallery,point_of_interest,establishment,NaN,NaN,51.492096,-0.118686,4.7,...,ChIJgVm1wuoEdkgR3rdQPOlrpvo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Enchanted Group,"Orange Studio, Old Paradise, 20 Carlisle Lane,...",art_gallery,cafe,electronics_store,home_goods_store,point_of_interest,51.498503,-0.116619,4.6,...,ChIJZ59JF78EdkgRM-4qe8X21Gw,food,store,establishment,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3635,Revolution Beaconsfield,"Maxwell Rd, Beaconsfield",bar,restaurant,food,point_of_interest,establishment,51.610356,-0.643456,3.8,...,ChIJRbM205dmdkgRbAq2iLexZFc,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
3636,Lizard Lounge,"186-188 George Ln, London",bar,restaurant,food,point_of_interest,establishment,51.594563,0.023954,4.1,...,ChIJAbbtCTan2EcRwrKLmaACj9U,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
3637,Popworld Guildford,"2-3 Armour Buildings, Guildford",night_club,bar,point_of_interest,establishment,NaN,51.236479,-0.577977,4.0,...,ChIJeVOZkJnQdUgRIZSNgDAyDq8,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
3638,64th & Social,"64 Clapham High St, London",bar,point_of_interest,establishment,NaN,NaN,51.463736,-0.133134,4.1,...,ChIJLS6ewFEEdkgRtjUDFZG1lXo,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN


In [8]:
DataV2.columns

Index(['name', 'vicinity', 'type/0', 'type/1', 'type/2', 'type/3', 'type/4',
       'geometry/location/lat', 'geometry/location/lng', 'rating',
       'user_ratings_total', 'place_id', 'type/5', 'type/6', 'type/7',
       'price_level', 'type/8', 'type/9', 'type/10', 'type/11', 'type/12'],
      dtype='object')

In [9]:
# Rearranging the columns in a new dataframe
DataV3 = df({
    'Name': DataV2.name,
    'Area': DataV2.vicinity,
    'Latitude': DataV2['geometry/location/lat'],
    'Longitude': DataV2['geometry/location/lng'],
    'Rating': DataV2.rating,
    'No_of_reviews': DataV2.user_ratings_total,
    'Place_ID': DataV2.place_id,
    'Price_level': DataV2.price_level,
    'Type_0': DataV2['type/0'],
    'Type_1': DataV2['type/1'],
    'Type_2': DataV2['type/2'],
    'Type_3': DataV2['type/3'],
    'Type_4': DataV2['type/4'],
    'Type_5': DataV2['type/5'],
    'Type_6': DataV2['type/6'],
    'Type_7': DataV2['type/7'],
    'Type_8': DataV2['type/8'],
    'Type_9': DataV2['type/9'],
    'Type_10': DataV2['type/10'],
    'Type_11': DataV2['type/11'],
    'Type_12': DataV2['type/12'],
})

Initial work on the reviews data

In [10]:
# Pickin the necessary columns for this dataset
ReviewsV2 = reviews[['result/formatted_address','result/name','result/formatted_phone_number',
                     'result/geometry/location/lat','result/geometry/location/lng','result/place_id',
                    'result/rating','result/user_ratings_total',
                    'result/reviews/0/text', 'result/reviews/0/rating',
                    'result/reviews/1/text', 'result/reviews/1/rating',
                    'result/reviews/2/text', 'result/reviews/2/rating',
                    'result/reviews/3/text', 'result/reviews/3/rating',
                    'result/reviews/4/text', 'result/reviews/4/rating']]
ReviewsV2

,result/formatted_address,result/name,result/formatted_phone_number,result/geometry/location/lat,result/geometry/location/lng,result/place_id,result/rating,result/user_ratings_total,result/reviews/0/text,result/reviews/0/rating,result/reviews/1/text,result/reviews/1/rating,result/reviews/2/text,result/reviews/2/rating,result/reviews/3/text,result/reviews/3/rating,result/reviews/4/text,result/reviews/4/rating
0,"Millbank, London SW1P 4RG, UK",Tate Britain,020 7887 8888,51.491062,-0.127789,ChIJvWjCxekEdkgRoCgQVJHZH_U,4.7,15422.0,It's free..! Loads of British pictures through...,5.0,Wonderful place to spend a few hours. So much ...,5.0,Visited the amazing Life Between Islands exhib...,5.0,"After up 8 floors in lift... Sorry, closed for...",1.0,Amazing to walk into the Hew Locke structure c...,5.0
1,"The Mall, St. James's, London SW1Y 5AH, UK",Institute of Contemporary Arts,020 7930 3647,51.506333,-0.130962,ChIJpwrHSNAEdkgR1YO1RjflYXw,4.4,1087.0,Very and helpful staff. Elijah particularly wa...,5.0,Awesome space that gives step to developing ar...,5.0,Great cinema. Fantastic space for exhibitions....,5.0,"Lovely bookstore, cafe and theater. I'd wager ...",5.0,Accidentally gave a 1 star review but I love t...,5.0
2,"16 John Islip St, London SW1P 4JU, UK",Institute of International Visual Arts,020 7630 1278,51.490483,-0.129298,ChIJrxGsi7ocdkgRfYUYz7mhuMs,5.0,7.0,You peaked my intrest.,5.0,The 5 Stars say it all.,5.0,Beautifully design space for which art to shine,5.0,New location in Pimlico.,5.0,good,5.0
3,"Beaconsfield Ltd, 22 Newport St, London SE11 6...",Beaconsfield Contemporary Art,020 7582 6465,51.492096,-0.118686,ChIJgVm1wuoEdkgR3rdQPOlrpvo,4.7,36.0,Great staff... location is not bad. Not allot ...,5.0,Fantastic,5.0,Wonderful gallery! Saw The Origin of Life by D...,5.0,An amazing place with wild hipster atmosphere ...,5.0,Great venue!,5.0
4,"Orange Studio, Old Paradise, 20 Carlisle Lane,...",The Enchanted Group,020 3468 9211,51.498503,-0.116619,ChIJZ59JF78EdkgRM-4qe8X21Gw,4.6,16.0,Nice vibes and cool energy. Everyone is very h...,5.0,Great atmos and great music in an eclectic bea...,5.0,A beautiful cultural oasis!,5.0,Really interesting multipurpose space. Went th...,5.0,Creative place. Relaxing space.,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3634,"Maxwell Rd, Beaconsfield HP9 1QX, UK",Revolution Beaconsfield,01494 677808,51.610356,-0.643456,ChIJRbM205dmdkgRbAq2iLexZFc,3.8,431.0,said it was 10-3 when in reality they rushed u...,2.0,Staff are always friendly. It's clean. It's we...,5.0,"Have tried to book a birthday party, gave up a...",1.0,"Well worth the £500 for the VIP booth, drinks ...",5.0,Nice and spacious staff were friendly and help...,5.0
3635,"186-188 George Ln, London E18 1AY, UK",Lizard Lounge,020 8989 3991,51.594557,0.023942,ChIJAbbtCTan2EcRwrKLmaACj9U,4.1,328.0,Paid £100 to reserve a table (non refundable) ...,1.0,"Very good service, 👍 cocktails, music, ambie...",5.0,"We've been several times, it's our local. It's...",5.0,"Great little bar with a warm atmosphere, good ...",5.0,"Great cocktails, great music, great atmosphere...",4.0
3636,"2-3 Armour Buildings, Guildford GU1 4SB, UK",Popworld Guildford,01483 306661,51.236479,-0.577977,ChIJeVOZkJnQdUgRIZSNgDAyDq8,4.0,164.0,Amazing night out. Music was on point with gre...,5.0,Most amazing night out in Guildford. Only plac...,5.0,Amazing night out! Staff were really helpful a...,5.0,"Staff are really friendly, went the extra mile...",5.0,"Always enjoy my nights out here, cheap drinks,...",5.0
3637,"64 Clapham High St, London SW4 7UL, UK",64th & Social,020 7627 2535,51.463736,-0.133134,ChIJLS6ewFEEdkgRtjUDFZG1lXo,4.1,232.0,I needed to use the bathroom and the toilet at...,1.0,Sixty Four is my favourite cocktail bar in Lon...,5.0,Felicia is the best and banging cocktails ❤️,5.0,Nice bar but the music is a complete let down....,1.0,Amazing bar and great service 👌,5.0


In [11]:
ReviewsV2 = ReviewsV2.dropna(axis=0, subset=['result/user_ratings_total'])
ReviewsV2.reset_index(drop=True, inplace=True)

In [12]:
# Rearrange the columns

ReviewsV3 = df({
    'Name': ReviewsV2['result/name'],
    'Address': ReviewsV2['result/formatted_address'],
    'Phone_Number': ReviewsV2['result/formatted_phone_number'],
    'Latitude': ReviewsV2['result/geometry/location/lat'],
    'Longitude': ReviewsV2['result/geometry/location/lng'],
    'Rating': ReviewsV2['result/rating'],
    'No_of_reviews': ReviewsV2['result/user_ratings_total'],
    'Place_ID': ReviewsV2['result/place_id'],
    'Review_1_text': ReviewsV2['result/reviews/0/text'],
    'Review_1_rate': ReviewsV2['result/reviews/0/rating'],
    'Review_2_text': ReviewsV2['result/reviews/1/text'],
    'Review_2_rate': ReviewsV2['result/reviews/1/rating'],
    'Review_3_text': ReviewsV2['result/reviews/2/text'],
    'Review_3_rate': ReviewsV2['result/reviews/2/rating'],
    'Review_4_text': ReviewsV2['result/reviews/3/text'],
    'Review_4_rate': ReviewsV2['result/reviews/3/rating'],
    'Review_5_text': ReviewsV2['result/reviews/4/text'],
    'Review_5_rate': ReviewsV2['result/reviews/4/rating']
})

ReviewsV3 = ReviewsV3.astype({'No_of_reviews': int}) # Fixing column type

In [13]:
ReviewsV3

,Name,Address,Phone_Number,Latitude,Longitude,Rating,No_of_reviews,Place_ID,Review_1_text,Review_1_rate,Review_2_text,Review_2_rate,Review_3_text,Review_3_rate,Review_4_text,Review_4_rate,Review_5_text,Review_5_rate
0,Tate Britain,"Millbank, London SW1P 4RG, UK",020 7887 8888,51.491062,-0.127789,4.7,15422,ChIJvWjCxekEdkgRoCgQVJHZH_U,It's free..! Loads of British pictures through...,5.0,Wonderful place to spend a few hours. So much ...,5.0,Visited the amazing Life Between Islands exhib...,5.0,"After up 8 floors in lift... Sorry, closed for...",1.0,Amazing to walk into the Hew Locke structure c...,5.0
1,Institute of Contemporary Arts,"The Mall, St. James's, London SW1Y 5AH, UK",020 7930 3647,51.506333,-0.130962,4.4,1087,ChIJpwrHSNAEdkgR1YO1RjflYXw,Very and helpful staff. Elijah particularly wa...,5.0,Awesome space that gives step to developing ar...,5.0,Great cinema. Fantastic space for exhibitions....,5.0,"Lovely bookstore, cafe and theater. I'd wager ...",5.0,Accidentally gave a 1 star review but I love t...,5.0
2,Institute of International Visual Arts,"16 John Islip St, London SW1P 4JU, UK",020 7630 1278,51.490483,-0.129298,5.0,7,ChIJrxGsi7ocdkgRfYUYz7mhuMs,You peaked my intrest.,5.0,The 5 Stars say it all.,5.0,Beautifully design space for which art to shine,5.0,New location in Pimlico.,5.0,good,5.0
3,Beaconsfield Contemporary Art,"Beaconsfield Ltd, 22 Newport St, London SE11 6...",020 7582 6465,51.492096,-0.118686,4.7,36,ChIJgVm1wuoEdkgR3rdQPOlrpvo,Great staff... location is not bad. Not allot ...,5.0,Fantastic,5.0,Wonderful gallery! Saw The Origin of Life by D...,5.0,An amazing place with wild hipster atmosphere ...,5.0,Great venue!,5.0
4,The Enchanted Group,"Orange Studio, Old Paradise, 20 Carlisle Lane,...",020 3468 9211,51.498503,-0.116619,4.6,16,ChIJZ59JF78EdkgRM-4qe8X21Gw,Nice vibes and cool energy. Everyone is very h...,5.0,Great atmos and great music in an eclectic bea...,5.0,A beautiful cultural oasis!,5.0,Really interesting multipurpose space. Went th...,5.0,Creative place. Relaxing space.,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3045,Revolution Beaconsfield,"Maxwell Rd, Beaconsfield HP9 1QX, UK",01494 677808,51.610356,-0.643456,3.8,431,ChIJRbM205dmdkgRbAq2iLexZFc,said it was 10-3 when in reality they rushed u...,2.0,Staff are always friendly. It's clean. It's we...,5.0,"Have tried to book a birthday party, gave up a...",1.0,"Well worth the £500 for the VIP booth, drinks ...",5.0,Nice and spacious staff were friendly and help...,5.0
3046,Lizard Lounge,"186-188 George Ln, London E18 1AY, UK",020 8989 3991,51.594557,0.023942,4.1,328,ChIJAbbtCTan2EcRwrKLmaACj9U,Paid £100 to reserve a table (non refundable) ...,1.0,"Very good service, 👍 cocktails, music, ambie...",5.0,"We've been several times, it's our local. It's...",5.0,"Great little bar with a warm atmosphere, good ...",5.0,"Great cocktails, great music, great atmosphere...",4.0
3047,Popworld Guildford,"2-3 Armour Buildings, Guildford GU1 4SB, UK",01483 306661,51.236479,-0.577977,4.0,164,ChIJeVOZkJnQdUgRIZSNgDAyDq8,Amazing night out. Music was on point with gre...,5.0,Most amazing night out in Guildford. Only plac...,5.0,Amazing night out! Staff were really helpful a...,5.0,"Staff are really friendly, went the extra mile...",5.0,"Always enjoy my nights out here, cheap drinks,...",5.0
3048,64th & Social,"64 Clapham High St, London SW4 7UL, UK",020 7627 2535,51.463736,-0.133134,4.1,232,ChIJLS6ewFEEdkgRtjUDFZG1lXo,I needed to use the bathroom and the toilet at...,1.0,Sixty Four is my favourite cocktail bar in Lon...,5.0,Felicia is the best and banging cocktails ❤️,5.0,Nice bar but the music is a complete let down....,1.0,Amazing bar and great service 👌,5.0


In [14]:
Name=[]
Address=[]
Number = []
Lat = []
Lng = []
Rate = []
ttlrate = []
pid = []
rev = []
revrate = []

for i in range(0,len(ReviewsV3)):
    rev.append(ReviewsV3['Review_1_text'][i])
    revrate.append(ReviewsV3['Review_1_rate'][i])
    rev.append(ReviewsV3['Review_2_text'][i])
    revrate.append(ReviewsV3['Review_2_rate'][i])
    rev.append(ReviewsV3['Review_3_text'][i])
    revrate.append(ReviewsV3['Review_3_rate'][i])    
    rev.append(ReviewsV3['Review_4_text'][i])
    revrate.append(ReviewsV3['Review_4_rate'][i])    
    rev.append(ReviewsV3['Review_5_text'][i])
    revrate.append(ReviewsV3['Review_5_rate'][i])
    for j in range(0,5):
        Name.append(ReviewsV3['Name'][i])
        Address.append(ReviewsV3['Address'][i])
        Number.append(ReviewsV3['Phone_Number'][i])
        Lat.append(ReviewsV3['Latitude'][i])
        Lng.append(ReviewsV3['Longitude'][i])
        Rate.append(ReviewsV3['Rating'][i])
        ttlrate.append(ReviewsV3['No_of_reviews'][i])
        pid.append(ReviewsV3['Place_ID'][i])    

In [15]:
ReviewsV4 = {'Name':Name, 'Address':Address, 'Phone_Number':Number, 'Latitude':Lat, 'Longitude':Lng, 'Rating':Rate,
             'No_of_reviews':ttlrate, 'Place_ID':pid, 'Review_text':rev,'Review_rate':revrate}

In [16]:
ReviewsV4 = df(ReviewsV4)
ReviewsV4 = ReviewsV4.dropna(axis=0, subset=['Review_rate'])
ReviewsV4 = ReviewsV4.dropna(axis=0, subset=['Review_text'])
ReviewsV4.reset_index(drop=True, inplace=True)

In [17]:
Places = DataV3
Reviews = ReviewsV4

In [18]:
Places

,Name,Area,Latitude,Longitude,Rating,No_of_reviews,Place_ID,Price_level,Type_0,Type_1,...,Type_3,Type_4,Type_5,Type_6,Type_7,Type_8,Type_9,Type_10,Type_11,Type_12
0,Tate Britain,"Millbank, London",51.491062,-0.127789,4.7,15400.0,ChIJvWjCxekEdkgRoCgQVJHZH_U,NaN,art_gallery,tourist_attraction,...,point_of_interest,establishment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Institute of Contemporary Arts,"The Mall, London",51.506333,-0.130962,4.4,1087.0,ChIJpwrHSNAEdkgR1YO1RjflYXw,NaN,art_gallery,tourist_attraction,...,point_of_interest,establishment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Institute of International Visual Arts,"16 John Islip Street, London",51.490483,-0.129298,5.0,7.0,ChIJrxGsi7ocdkgRfYUYz7mhuMs,NaN,art_gallery,library,...,establishment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Beaconsfield Contemporary Art,"Beaconsfield Ltd, 22 Newport Street, London",51.492096,-0.118686,4.7,36.0,ChIJgVm1wuoEdkgR3rdQPOlrpvo,NaN,art_gallery,point_of_interest,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Enchanted Group,"Orange Studio, Old Paradise, 20 Carlisle Lane,...",51.498503,-0.116619,4.6,16.0,ChIJZ59JF78EdkgRM-4qe8X21Gw,NaN,art_gallery,cafe,...,home_goods_store,point_of_interest,food,store,establishment,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3635,Revolution Beaconsfield,"Maxwell Rd, Beaconsfield",51.610356,-0.643456,3.8,430.0,ChIJRbM205dmdkgRbAq2iLexZFc,2.0,bar,restaurant,...,point_of_interest,establishment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3636,Lizard Lounge,"186-188 George Ln, London",51.594563,0.023954,4.1,327.0,ChIJAbbtCTan2EcRwrKLmaACj9U,2.0,bar,restaurant,...,point_of_interest,establishment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3637,Popworld Guildford,"2-3 Armour Buildings, Guildford",51.236479,-0.577977,4.0,158.0,ChIJeVOZkJnQdUgRIZSNgDAyDq8,2.0,night_club,bar,...,establishment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3638,64th & Social,"64 Clapham High St, London",51.463736,-0.133134,4.1,232.0,ChIJLS6ewFEEdkgRtjUDFZG1lXo,2.0,bar,point_of_interest,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
Reviews

,Name,Address,Phone_Number,Latitude,Longitude,Rating,No_of_reviews,Place_ID,Review_text,Review_rate
0,Tate Britain,"Millbank, London SW1P 4RG, UK",020 7887 8888,51.491062,-0.127789,4.7,15422,ChIJvWjCxekEdkgRoCgQVJHZH_U,It's free..! Loads of British pictures through...,5.0
1,Tate Britain,"Millbank, London SW1P 4RG, UK",020 7887 8888,51.491062,-0.127789,4.7,15422,ChIJvWjCxekEdkgRoCgQVJHZH_U,Wonderful place to spend a few hours. So much ...,5.0
2,Tate Britain,"Millbank, London SW1P 4RG, UK",020 7887 8888,51.491062,-0.127789,4.7,15422,ChIJvWjCxekEdkgRoCgQVJHZH_U,Visited the amazing Life Between Islands exhib...,5.0
3,Tate Britain,"Millbank, London SW1P 4RG, UK",020 7887 8888,51.491062,-0.127789,4.7,15422,ChIJvWjCxekEdkgRoCgQVJHZH_U,"After up 8 floors in lift... Sorry, closed for...",1.0
4,Tate Britain,"Millbank, London SW1P 4RG, UK",020 7887 8888,51.491062,-0.127789,4.7,15422,ChIJvWjCxekEdkgRoCgQVJHZH_U,Amazing to walk into the Hew Locke structure c...,5.0
...,...,...,...,...,...,...,...,...,...,...
13378,The Hive Bar,"53 High St, Crawley RH10 1BQ, UK",01293 611313,51.114700,-0.190071,3.9,167,ChIJhVmfPB_udUgRpfI_49k9-Mc,Nice Bar friendly staff but prices are steep,4.0
13379,The Hive Bar,"53 High St, Crawley RH10 1BQ, UK",01293 611313,51.114700,-0.190071,3.9,167,ChIJhVmfPB_udUgRpfI_49k9-Mc,"Great service, great cocktails",5.0
13380,The Hive Bar,"53 High St, Crawley RH10 1BQ, UK",01293 611313,51.114700,-0.190071,3.9,167,ChIJhVmfPB_udUgRpfI_49k9-Mc,Great atmosphere and lovely staff,5.0
13381,The Hive Bar,"53 High St, Crawley RH10 1BQ, UK",01293 611313,51.114700,-0.190071,3.9,167,ChIJhVmfPB_udUgRpfI_49k9-Mc,"Amazing, bar staff were so friendly and good d...",5.0


In [20]:
print(Places.isnull().sum(),'\n')
print(Reviews.isnull().sum(),'\n')

Name                0
Area               17
Latitude            0
Longitude           0
Rating            549
No_of_reviews     549
Place_ID            0
Price_level      2705
Type_0              0
Type_1              0
Type_2            149
Type_3           1586
Type_4           2591
Type_5           3218
Type_6           3492
Type_7           3581
Type_8           3616
Type_9           3626
Type_10          3632
Type_11          3633
Type_12          3636
dtype: int64 

Name                0
Address             0
Phone_Number     2002
Latitude            0
Longitude           0
Rating              0
No_of_reviews       0
Place_ID            0
Review_text         0
Review_rate         0
dtype: int64 



In [21]:
Places.to_csv('Places.csv')
Reviews.to_csv('Reviews.csv')

In [27]:
Places[['Rating','No_of_reviews','Price_level']].describe().apply(lambda s: s.apply('{0:.1f}'.format))

,Rating,No_of_reviews,Price_level
count,3090.0,3090.0,934.0
mean,4.2,1005.6,2.0
std,0.7,6391.3,0.6
min,0.0,0.0,1.0
25%,4.0,15.0,2.0
50%,4.3,110.0,2.0
75%,4.6,540.8,2.0
max,5.0,146161.0,4.0


In [28]:
Reviews[['Rating','No_of_reviews','Review_rate']].describe().apply(lambda s: s.apply('{0:.1f}'.format))

,Rating,No_of_reviews,Review_rate
count,13383.0,13383.0,13383.0
mean,4.2,1164.0,4.3
std,0.5,6870.0,1.3
min,1.0,1.0,1.0
25%,4.0,34.0,4.0
50%,4.3,177.0,5.0
75%,4.6,655.0,5.0
max,5.0,146460.0,5.0


In [30]:
Merged = Places.merge(Reviews, how = 'inner', on = ['Place_ID'])

In [31]:
Merged = Merged.drop_duplicates(subset = 'Place_ID')
Merged = Merged.drop(['Review_text','Review_rate','Latitude_x','Longitude_x','Rating_x','No_of_reviews_x'], axis=1)
Merged

,Name_x,Area,Place_ID,Price_level,Type_0,Type_1,Type_2,Type_3,Type_4,Type_5,...,Type_10,Type_11,Type_12,Name_y,Address,Phone_Number,Latitude_y,Longitude_y,Rating_y,No_of_reviews_y
0,Tate Britain,"Millbank, London",ChIJvWjCxekEdkgRoCgQVJHZH_U,NaN,art_gallery,tourist_attraction,museum,point_of_interest,establishment,NaN,...,NaN,NaN,NaN,Tate Britain,"Millbank, London SW1P 4RG, UK",020 7887 8888,51.491062,-0.127789,4.7,15422
5,Institute of Contemporary Arts,"The Mall, London",ChIJpwrHSNAEdkgR1YO1RjflYXw,NaN,art_gallery,tourist_attraction,movie_theater,point_of_interest,establishment,NaN,...,NaN,NaN,NaN,Institute of Contemporary Arts,"The Mall, St. James's, London SW1Y 5AH, UK",020 7930 3647,51.506333,-0.130962,4.4,1087
10,Institute of International Visual Arts,"16 John Islip Street, London",ChIJrxGsi7ocdkgRfYUYz7mhuMs,NaN,art_gallery,library,point_of_interest,establishment,NaN,NaN,...,NaN,NaN,NaN,Institute of International Visual Arts,"16 John Islip St, London SW1P 4JU, UK",020 7630 1278,51.490483,-0.129298,5.0,7
15,Beaconsfield Contemporary Art,"Beaconsfield Ltd, 22 Newport Street, London",ChIJgVm1wuoEdkgR3rdQPOlrpvo,NaN,art_gallery,point_of_interest,establishment,NaN,NaN,NaN,...,NaN,NaN,NaN,Beaconsfield Contemporary Art,"Beaconsfield Ltd, 22 Newport St, London SE11 6...",020 7582 6465,51.492096,-0.118686,4.7,36
20,The Enchanted Group,"Orange Studio, Old Paradise, 20 Carlisle Lane,...",ChIJZ59JF78EdkgRM-4qe8X21Gw,NaN,art_gallery,cafe,electronics_store,home_goods_store,point_of_interest,food,...,NaN,NaN,NaN,The Enchanted Group,"Orange Studio, Old Paradise, 20 Carlisle Lane,...",020 3468 9211,51.498503,-0.116619,4.6,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13358,Revolution Beaconsfield,"Maxwell Rd, Beaconsfield",ChIJRbM205dmdkgRbAq2iLexZFc,2.0,bar,restaurant,food,point_of_interest,establishment,NaN,...,NaN,NaN,NaN,Revolution Beaconsfield,"Maxwell Rd, Beaconsfield HP9 1QX, UK",01494 677808,51.610356,-0.643456,3.8,431
13363,Lizard Lounge,"186-188 George Ln, London",ChIJAbbtCTan2EcRwrKLmaACj9U,2.0,bar,restaurant,food,point_of_interest,establishment,NaN,...,NaN,NaN,NaN,Lizard Lounge,"186-188 George Ln, London E18 1AY, UK",020 8989 3991,51.594557,0.023942,4.1,328
13368,Popworld Guildford,"2-3 Armour Buildings, Guildford",ChIJeVOZkJnQdUgRIZSNgDAyDq8,2.0,night_club,bar,point_of_interest,establishment,NaN,NaN,...,NaN,NaN,NaN,Popworld Guildford,"2-3 Armour Buildings, Guildford GU1 4SB, UK",01483 306661,51.236479,-0.577977,4.0,164
13373,64th & Social,"64 Clapham High St, London",ChIJLS6ewFEEdkgRtjUDFZG1lXo,2.0,bar,point_of_interest,establishment,NaN,NaN,NaN,...,NaN,NaN,NaN,64th & Social,"64 Clapham High St, London SW4 7UL, UK",020 7627 2535,51.463736,-0.133134,4.1,232


In [32]:
Merged.reset_index(drop=True, inplace=True)
Merged.columns

Index(['Name_x', 'Area', 'Place_ID', 'Price_level', 'Type_0', 'Type_1',
       'Type_2', 'Type_3', 'Type_4', 'Type_5', 'Type_6', 'Type_7', 'Type_8',
       'Type_9', 'Type_10', 'Type_11', 'Type_12', 'Name_y', 'Address',
       'Phone_Number', 'Latitude_y', 'Longitude_y', 'Rating_y',
       'No_of_reviews_y'],
      dtype='object')

In [33]:
PlacesV2 =  df({
    'Name': Merged.Name_x,
    'Area': Merged.Area,
    'Address': Merged.Address,
    'Latitude': Merged.Latitude_y,
    'Longitude': Merged.Longitude_y,
    'Price_level': Merged.Price_level,
    'Rating': Merged.Rating_y,
    'No_of_reviews': Merged.No_of_reviews_y,
    'Type_0': Merged.Type_0 , 
    'Type_1': Merged.Type_1 ,
    'Type_2': Merged.Type_2 , 
    'Type_3': Merged.Type_3 , 
    'Phone_Number': Merged.Phone_Number,
    'Place_ID': Merged.Place_ID
})

PlacesV2

,Name,Area,Address,Latitude,Longitude,Price_level,Rating,No_of_reviews,Type_0,Type_1,Type_2,Type_3,Phone_Number,Place_ID
0,Tate Britain,"Millbank, London","Millbank, London SW1P 4RG, UK",51.491062,-0.127789,NaN,4.7,15422,art_gallery,tourist_attraction,museum,point_of_interest,020 7887 8888,ChIJvWjCxekEdkgRoCgQVJHZH_U
1,Institute of Contemporary Arts,"The Mall, London","The Mall, St. James's, London SW1Y 5AH, UK",51.506333,-0.130962,NaN,4.4,1087,art_gallery,tourist_attraction,movie_theater,point_of_interest,020 7930 3647,ChIJpwrHSNAEdkgR1YO1RjflYXw
2,Institute of International Visual Arts,"16 John Islip Street, London","16 John Islip St, London SW1P 4JU, UK",51.490483,-0.129298,NaN,5.0,7,art_gallery,library,point_of_interest,establishment,020 7630 1278,ChIJrxGsi7ocdkgRfYUYz7mhuMs
3,Beaconsfield Contemporary Art,"Beaconsfield Ltd, 22 Newport Street, London","Beaconsfield Ltd, 22 Newport St, London SE11 6...",51.492096,-0.118686,NaN,4.7,36,art_gallery,point_of_interest,establishment,NaN,020 7582 6465,ChIJgVm1wuoEdkgR3rdQPOlrpvo
4,The Enchanted Group,"Orange Studio, Old Paradise, 20 Carlisle Lane,...","Orange Studio, Old Paradise, 20 Carlisle Lane,...",51.498503,-0.116619,NaN,4.6,16,art_gallery,cafe,electronics_store,home_goods_store,020 3468 9211,ChIJZ59JF78EdkgRM-4qe8X21Gw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2916,Revolution Beaconsfield,"Maxwell Rd, Beaconsfield","Maxwell Rd, Beaconsfield HP9 1QX, UK",51.610356,-0.643456,2.0,3.8,431,bar,restaurant,food,point_of_interest,01494 677808,ChIJRbM205dmdkgRbAq2iLexZFc
2917,Lizard Lounge,"186-188 George Ln, London","186-188 George Ln, London E18 1AY, UK",51.594557,0.023942,2.0,4.1,328,bar,restaurant,food,point_of_interest,020 8989 3991,ChIJAbbtCTan2EcRwrKLmaACj9U
2918,Popworld Guildford,"2-3 Armour Buildings, Guildford","2-3 Armour Buildings, Guildford GU1 4SB, UK",51.236479,-0.577977,2.0,4.0,164,night_club,bar,point_of_interest,establishment,01483 306661,ChIJeVOZkJnQdUgRIZSNgDAyDq8
2919,64th & Social,"64 Clapham High St, London","64 Clapham High St, London SW4 7UL, UK",51.463736,-0.133134,2.0,4.1,232,bar,point_of_interest,establishment,NaN,020 7627 2535,ChIJLS6ewFEEdkgRtjUDFZG1lXo


In [43]:
PlacesV2[['Price_level','Rating','No_of_reviews']].describe().apply(lambda s: s.apply('{0:.1f}'.format))

,Price_level,Rating,No_of_reviews
count,930.0,2921.0,2921.0
mean,2.0,4.3,1066.9
std,0.6,0.6,6585.1
min,1.0,1.0,1.0
25%,2.0,4.0,22.0
50%,2.0,4.3,135.0
75%,2.0,4.6,582.0
max,4.0,5.0,146460.0


In [29]:
PlacesV2.to_csv('PlacesV2.csv')